<a href="https://colab.research.google.com/github/emiree-f/CS461/blob/main/CS461Program3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Dense

In [2]:
filename = '/content/StudentsPerformance.csv'

In [3]:
data = pd.read_csv(filename)
data.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
print(data.shape)

(1000, 8)


In [5]:
# Use a 70/15/15 split for Training/Test/Validation data
train_size = 0.7

# Last 3 columns are our target columns
X = data.drop(columns=['math score','reading score','writing score']).copy()
y = data[data.columns.values[-3:]].copy()

print(X.head())
print(y.head())

   gender race/ethnicity  ...         lunch test preparation course
0  female        group B  ...      standard                    none
1  female        group C  ...      standard               completed
2  female        group B  ...      standard                    none
3    male        group A  ...  free/reduced                    none
4    male        group C  ...      standard                    none

[5 rows x 5 columns]
   math score  reading score  writing score
0          72             72             74
1          69             90             88
2          90             95             93
3          47             57             44
4          76             78             75


One-hot encode categorical input variables

In [6]:
# Check columns to encode
for col in X.columns:
  print(X[col].value_counts())
  print()

female    518
male      482
Name: gender, dtype: int64

group C    319
group D    262
group B    190
group E    140
group A     89
Name: race/ethnicity, dtype: int64

some college          226
associate's degree    222
high school           196
some high school      179
bachelor's degree     118
master's degree        59
Name: parental level of education, dtype: int64

standard        645
free/reduced    355
Name: lunch, dtype: int64

none         642
completed    358
Name: test preparation course, dtype: int64



In [7]:
# One-Hot encoding the categorical parameters using get_dummies()
one_hot_encoded_X = pd.get_dummies(X, columns = X.columns)
one_hot_encoded_X.head()

,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none
0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1
1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0
2,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1
3,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1
4,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1


In [8]:
# Split off training set
X_train, X_rem, y_train, y_rem = train_test_split(one_hot_encoded_X, y, train_size=train_size)

print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)
print("X_rem.shape: ", X_rem.shape)
print("y_rem.shape: ", y_rem.shape)

X_train.shape:  (700, 17)
y_train.shape:  (700, 3)
X_rem.shape:  (300, 17)
y_rem.shape:  (300, 3)


In [9]:
test_size = 0.5
# Split test and validation sets
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=test_size)

print("X_valid.shape: ", X_valid.shape)
print("y_valid.shape: ", y_valid.shape)
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ", y_test.shape)

X_valid.shape:  (150, 17)
y_valid.shape:  (150, 3)
X_test.shape:  (150, 17)
y_test.shape:  (150, 3)


In [10]:
n_inputs = one_hot_encoded_X.shape[1]
n_outputs = y.shape[1]

In [11]:
# Create model and add layers
model = Sequential()
model.add(Dense(20, activation='relu', input_dim=n_inputs))
model.add(Dense(n_outputs, activation='linear'))
model.compile(loss='mae', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                360       
                                                                 
 dense_1 (Dense)             (None, 3)                 63        
                                                                 
Total params: 423
Trainable params: 423
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Fit model on training set
history = model.fit(X_train, y_train, verbose=0, epochs=2000)

In [13]:
# Fit model on validation set
history = model.fit(X_valid, y_valid, verbose=0, epochs=2000)

In [14]:
# Evaluate model on test set
model.evaluate(X_test,y_test)

5/5 [==============================] - 0s 4ms/step - loss: 10.9376 - accuracy: 0.5267


[10.937607765197754, 0.5266666412353516]